In [53]:
import cv2 as cv

image = cv.imread("../data/images/road818.png")

In [4]:
def show_img(img, title:str="Image"):
    cv.imshow(title, img)
    cv.waitKey(0)
    cv.destroyWindow(title)

In [8]:
def contrast(bgr_image):
    ycrcb_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2YCrCb)
    
    y, cr, cb = cv.split(ycrcb_image)
    y_equalized = cv.equalizeHist(y)
    equalized_image = cv.merge([y_equalized, cr, cb])
    return cv.cvtColor(equalized_image, cv.COLOR_YCrCb2BGR)

In [24]:
def automatic_brightness_contrast(bgr_image, clip_hist_percent = 0.01, use_scale_abs = True, return_verbose = False):
    gray_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2GRAY)

    # Grayscale histogram of the image
    hist = cv.calcHist([gray_image], [0], None, [256], [0, 256])
    hist_size = len(hist)

    # Cumulative distribution of the histogram
    acc = []
    acc.append( float(hist[0]) )
    for i in range(1, hist_size):
        acc.append( acc[i - 1] + float(hist[i]) )
    
    # Locate points to clip
    maximum = acc[-1]
    clip_hist = clip_hist_percent * maximum / 2.0

    # Left cut
    minimum_gray = 0
    while acc[minimum_gray] < clip_hist:
        minimum_gray += 1

    # Right cut
    maximum_gray = hist_size - 1
    while acc[maximum_gray] >= (maximum - clip_hist):
        maximum_gray -= 1

    # Calculate alpha and beta values for the scaling
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = - minimum_gray * alpha

    if use_scale_abs:
        processed_image = cv.convertScaleAbs(bgr_image, alpha=alpha, beta=beta)
    else:
        processed_image = bgr_image * alpha + beta
        processed_image[processed_image < 0] = 0
        processed_image[processed_image > 255] = 255

    if return_verbose:
        processed_hist = cv.calcHist([gray_image], [0], None, [256], [minimum_gray, maximum_gray])

        return processed_image, alpha, beta, hist, processed_hist
    
    return processed_image

In [34]:
def LaplacianOfGaussian(bgr_image):
    log_image = cv.GaussianBlur(bgr_image, (3, 3), 0)
    gray_image = cv.cvtColor(log_image, cv.COLOR_BGR2GRAY)
    log_image = cv.Laplacian(log_image, cv.CV_8U, ksize=3, scale=1, delta=0)
    return cv.convertScaleAbs(log_image)

In [62]:
show_img(image)

new_img = automatic_brightness_contrast(image, clip_hist_percent=0.1, use_scale_abs=True)

#new_img = LaplacianOfGaussian(new_img)
show_img(new_img)

In [46]:
def extract_red_hsv(hsv_image, return_split = False):
    # First zone of reds
    lowerbound_1 = np.array([0, 40, 25])
    upperbound_1 = np.array([10, 255, 255])

    # Second zone of reds
    lowerbound_2 = np.array([135, 40, 25])
    upperbound_2 = np.array([179, 255, 255])

    red_1 = cv.inRange(hsv_image, lowerbound_1, upperbound_1)
    red_2 = cv.inRange(hsv_image, lowerbound_2, upperbound_2)

    if return_split:
        return red_1, red_2
    
    return cv.bitwise_or(red_1, red_2)

def extract_blue_hsv(hsv_image):
    lowerbound = np.array([100, 160, 40])
    upperbound = np.array([120, 255, 255])

    return cv.inRange(hsv_image, lowerbound, upperbound)

In [47]:
def binarize(bgr_image, threshold_percent = 0.75, return_thresh = False):
    gray_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2GRAY)

    threshold = int(threshold_percent * 255)

    thresh, thresh_image = cv.threshold(gray_image, threshold, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    if return_thresh:
        return thresh_image, thresh
    return thresh_image

In [105]:
def extractObjects(bgr_image, contours):
    out_image = np.zeros_like(bgr_image)

    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        out_image[y:y+h, x:x+w] = bgr_image[y:y+h, x:x+w]
    
    return out_image

In [104]:
show_img(red_img)

In [106]:
objects = extractObjects(image, contours)

In [119]:
contrast_image = automatic_brightness_contrast(objects, 0.05)

In [129]:
red_mask = extract_red_hsv(objects)

white_mask = cv.inRange(cv.cvtColor(objects, cv.COLOR_BGR2HSV), np.array([0, 0, 130]), np.array([255, 40, 255]))

mask = cv.bitwise_or(red_mask, white_mask)

cropped_objects = cv.bitwise_and(objects, objects, mask=mask)
show_img(objects)
show_img(cropped_objects)

bin_image = binarize(cropped_objects)

In [127]:
show_img(bin_image)
kernel = np.ones((3, 3), np.uint8)
morph_img = cv.morphologyEx(bin_image, cv.MORPH_DILATE, kernel)
show_img(morph_img)

In [ ]:
# WARMING FILTER
# https://towardsdatascience.com/python-opencv-building-instagram-like-image-filters-5c482c1c5079

def _create_LUT_8UC1(x, y):
  spl = UnivariateSpline(x, y)
  return spl(range(256))

# TODO better lut values for reds
incr_ch_lut = _create_LUT_8UC1([0, 64, 128, 256],[0, 80, 160, 256])
decr_ch_lut = _create_LUT_8UC1([0, 64, 128, 256],[0, 50,  100, 255])

def render(img_rgb):
    c_r, c_g, c_b = cv.split(img_rgb)
    c_r = cv.LUT(c_r, incr_ch_lut).astype(np.uint8)
    c_b = cv.LUT(c_b, decr_ch_lut).astype(np.uint8)
  
    return  cv.merge((c_r, c_g, c_b)) 

bla = cv.cvtColor(src_image, cv.COLOR_BGR2RGB)  
display_bgr_image(render(bla))

src_image = bla